In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
import gensim
import os
import seaborn as sns

In [3]:
train = pd.read_csv("../data/train_with_label.csv")

In [8]:
train["words"] = train["product_name"].apply(lambda words : [word for word in words.lower().replace(",", "").replace("&", "").split(" ") if word != ""])
train["bigram"] = train.words.apply(lambda words: [f'{words[i]} {words[i+1]}' for i in range(len(words)-1)])
train["trigram"] = train.words.apply(lambda words: [f'{words[i]} {words[i+1]} {words[i+2]}' for i in range(len(words)-2)])

In [13]:
from collections import Counter, defaultdict
counter = defaultdict(Counter)

for e in train.itertuples():
    counter[e.department_id].update([word for word in e.bigram if word != ""])
        
keywords = {}

for i in range(21):
    mc = counter[i].most_common(19)
    keywords[i] = [c[0] for c in mc]
    print(f"{i}: {mc}\n")

0: [('ice cream', 244), ('gluten free', 87), ('frozen dessert', 44), ('pepperoni pizza', 33), ('chocolate chip', 30), ('cream bars', 30), ('chicken breast', 28), ('thin crust', 26), ('cheese pizza', 26), ('vanilla ice', 23), ('peanut butter', 19), ('chocolate ice', 19), ('all natural', 17), ('dark chocolate', 17), ('white meat', 16), ('slow churned', 16), ('four cheese', 16), ('sea salt', 16), ('frozen dairy', 16)]

1: [('hand soap', 9), ('sleep aid', 8), ('diaper rash', 5), ('nighttime sleep', 5), ('cherry flavor', 5), ('latex condoms', 5), ('bloody mary', 5), ('1 mg', 4), ('personal lubricant', 4), ('natural calm', 4), ('rash ointment', 4), ('100% pure', 4), ('mg tablets', 4), ('mary mix', 4), ('melatonin 1', 3), ('beef chuck', 3), ('tea tree', 3), ('tree oil', 3), ('margarita mix', 3)]

2: [('whole wheat', 52), ('gluten free', 43), ('whole grain', 39), ('hamburger buns', 27), ('100% whole', 23), ('grain bread', 21), ('wheat bread', 21), ('hot dog', 18), ('flour tortillas', 18), ('en

In [14]:
print(keywords)

{0: ['ice cream', 'gluten free', 'frozen dessert', 'pepperoni pizza', 'chocolate chip', 'cream bars', 'chicken breast', 'thin crust', 'cheese pizza', 'vanilla ice', 'peanut butter', 'chocolate ice', 'all natural', 'dark chocolate', 'white meat', 'slow churned', 'four cheese', 'sea salt', 'frozen dairy'], 1: ['hand soap', 'sleep aid', 'diaper rash', 'nighttime sleep', 'cherry flavor', 'latex condoms', 'bloody mary', '1 mg', 'personal lubricant', 'natural calm', 'rash ointment', '100% pure', 'mg tablets', 'mary mix', 'melatonin 1', 'beef chuck', 'tea tree', 'tree oil', 'margarita mix'], 2: ['whole wheat', 'gluten free', 'whole grain', 'hamburger buns', '100% whole', 'grain bread', 'wheat bread', 'hot dog', 'flour tortillas', 'english muffins', 'dog buns', 'rye bread', 'white bread', 'pizza crust', 'corn tortillas', 'italian bread', 'soft taco', 'sliced bread', 'cinnamon raisin'], 3: ['salad kit', 'organic red', 'organic baby', 'bag of', 'organic green', 'spring mix', 'chopped salad', 'ba